<a href="https://colab.research.google.com/github/anitanadvikova/ScrabbleGAN_RU/blob/main/%20train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/arshjot/ScrabbleGAN.git

Cloning into 'ScrabbleGAN'...
remote: Enumerating objects: 208, done.
remote: Counting objects: 100% (208/208), done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 208 (delta 89), reused 173 (delta 57), pack-reused 0
Receiving objects: 100% (208/208), 202.94 KiB | 15.61 MiB/s, done.
Resolving deltas: 100% (89/89), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
#!unzip drive/MyDrive/RIMES.zip

1.   Перетащить RIMES в data
2.   Изменить конфиг

```python
lexicon_file = f'/content/drive/MyDrive/words_russian_5000.txt' #строка 23
data_folder_path ='/content/ScrabbleGAN/data/RIMES/'. #строка 6
num_chars = 108 #строка 51
```

3.   Изменить **data/prepare_data** строки 123-149

In [ ]:
# create char_map using training labels
with open(f'{data_folder_path}/ground_truth_training_icdar2011.tsv', 'rb') as f:
  ids = f.read().decode('utf8')
  partition_ids = [i.split()[0] for i in ids.splitlines() if len(i) > 1]
  words_raw = [(' '.join(i.split()[1:])) for i in ids.splitlines() if len(i)> 1]
# Get list of unique characters and create dictionary for mapping them to integer
chars=[]
for w_i in words_raw:
  for i in w_i:
    chars.append(i)
chars=np.unique(chars)

chars=np.unique(chars)
char_map = {value: idx + 1 for (idx, value) in enumerate(chars)}
char_map['<BLANK>'] = 0
num_chars = len(char_map.keys())
  
# Extract IDs for required set
with open(f'{data_folder_path}/ground_truth_{partition_name}_icdar2011.tsv', 'rb') as f:
  ids = f.read().decode('utf8')
  partition_ids = [i.split()[0] for i in ids.splitlines() if len(i) > 1]
  words_raw = [(' '.join(i.split()[1:])) for i in ids.splitlines() if len(i)> 1]

word_data = {}
for img_path, label in zip(partition_ids, words_raw):
    if len(label)!=0:
      img_path = f'{data_folder_path}/{partition_name}/{img_path}'
      img, valid_img = read_image(img_path, len(label), img_h, char_w)
      img_id = img_path[img_path.rfind('/')+1:-5]
      if valid_img:
          try:
              word_data[img_id] = [[char_map[char] for char in label], img]
          except KeyError:
              pass

In [16]:
%cd ScrabbleGAN/data

/content/ScrabbleGAN/data


In [5]:
!python prepare_data.py

Processing Data:

Number of images = 17913
Number of unique characters = 108

Data processing completed


In [19]:
!zip -r "/content/drive/MyDrive/RIMES_tr_data.zip" "RIMES_tr_data.pkl"

  adding: RIMES_tr_data.pkl (deflated 31%)


In [20]:
%cd ..

/content/ScrabbleGAN


Меняем `models/ScrabbleGAN.py`

```python
#строки 78-87
fake_words = []
        
with open(cfg.lexicon_file, 'r') as f:
          fake_words.extend(f.read().splitlines())
        
self.fake_words = fake_words 
```

In [ ]:
!python train.py

 Loading Data 
 Model: ScrabbleGAN 
NumExpr defaulting to 2 threads.
Param count for Gs initialized parameters: 30869377
Param count for Ds initialized parameters: 36398401
ScrabbleGAN(
  (R): Recognizer(
    (convs): Sequential(
      (0): Sequential(
        (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (1): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
        (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (2): Sequential(
        (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (3): Sequential(
        (0): Conv2d(256, 256, kernel_size=(3, 3),